In [1]:
# Create New Conda Environment and Use Conda Channel 
!pip install git+https://github.com/huggingface/parler-tts.git
!pip install transformers torch soundfile sklearn
!pip install openai



  Cloning https://github.com/huggingface/parler-tts.git to /tmp/pip-req-build-mw37wsdm
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/parler-tts.git /tmp/pip-req-build-mw37wsdm
  Resolved https://github.com/huggingface/parler-tts.git to commit d108732cd57788ec86bc857d99a6cabd66663d68
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/descriptinc/audiotools to /tmp/pip-install-9gxcazqt/descript-audiotools_e008f5b9bb56453ea48c1c251426312b
  Running command git clone --filter=blob:none --quiet https://github.com/descriptinc/audiotools /tmp/pip-install-9gxcazqt/descript-audiotools_e008f5b9bb56453ea48c1c251426312b
  Resolved https://github.com/descriptinc/audiotools to commit 348ebf2034ce24e2a91a553e3171cb00c0c71678
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.0 MB/s eta 0:00:00
  Prep

In [2]:
import torch
import torch.nn.functional as F
import soundfile as sf
import sys
import os
from parler_tts import ParlerTTSForConditionalGeneration
from transformers import AutoTokenizer

In [3]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

dataset_path = "/kaggle/input/llangdetect3/" 
sys.path.append(os.path.abspath(dataset_path))
from llama_ld import detect_languages

In [4]:
model = ParlerTTSForConditionalGeneration.from_pretrained("ai4bharat/indic-parler-tts").to(device)
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-parler-tts")
description_tokenizer = AutoTokenizer.from_pretrained(model.config.text_encoder._name_or_path)

def detect_language_with_fallback(text):
    try:
        word_languages = detect_languages(text)  # Can handle single words or sentences
        return word_languages if isinstance(word_languages, dict) else {word: "hi" for word in text.split()}
    except Exception as e:
        print(f"Error in language detection: {e}")
        return {word: "hi" for word in text.split()}  # Default to Hindi



config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.75G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/223 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/990 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [5]:
sentence = "Aaj ravivaar hai"

word_languages = detect_language_with_fallback(sentence)



API Response: ChatCompletion(id='chat-ced817ca413242ca843fe58e1549eeca', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{\n  "Aaj": "Hindi",\n  "ravivaar": "Hindi",\n  "hai": "Hindi"\n}', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None), stop_reason=None)], created=1742376781, model='meta/llama-3.1-8b-instruct', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=29, prompt_tokens=68, total_tokens=97, completion_tokens_details=None, prompt_tokens_details=None), prompt_logprobs=None)
JSON loaded successfully.


In [6]:
base_path = "../input/tts-dataset-hindi-accent/"
wav_files = [f for f in os.listdir(base_path) if f.endswith('.wav')]


In [7]:
def split_dataset(sentences):
    train_data, temp_data = train_test_split(sentences, test_size=0.2, random_state=42)
    
    val_data, test_data = train_test_split(temp_data, test_size=0.8, random_state=42)
    return train_data, val_data, test_data

In [8]:
processed_text = []
for word, lang in word_languages.items():
    print(f"Word: {word}, Lang: {lang}")  # Debugging
    description = "Speak this word in Hindi with a male Indian accent and a clear voice." if lang.lower() == "hindi" else "Speak this word in English with a female Indian accent and a clear voice."
    processed_text.append((word, description))


    processed_text.append((word, description))


print(word_languages)  # Verify key-value pairs


print(processed_text)  


Word: Aaj, Lang: Hindi
Word: ravivaar, Lang: Hindi
Word: hai, Lang: Hindi
{'Aaj': 'Hindi', 'ravivaar': 'Hindi', 'hai': 'Hindi'}
[('Aaj', 'Speak this word in Hindi with a male Indian accent and a clear voice.'), ('Aaj', 'Speak this word in Hindi with a male Indian accent and a clear voice.'), ('ravivaar', 'Speak this word in Hindi with a male Indian accent and a clear voice.'), ('ravivaar', 'Speak this word in Hindi with a male Indian accent and a clear voice.'), ('hai', 'Speak this word in Hindi with a male Indian accent and a clear voice.'), ('hai', 'Speak this word in Hindi with a male Indian accent and a clear voice.')]


In [9]:
audio_segments = []
for word, description in processed_text:
    description_inputs = description_tokenizer(description, return_tensors="pt").to(device)
    prompt_inputs = tokenizer(word, return_tensors="pt").to(device)  
    generation = model.generate(
        input_ids=description_inputs.input_ids,
        attention_mask=description_inputs.attention_mask,
        prompt_input_ids=prompt_inputs.input_ids,
        prompt_attention_mask=prompt_inputs.attention_mask
    )
    
    audio_segments.append(generation.cpu().numpy().squeeze())



In [10]:
final_audio = torch.from_numpy(audio_segments[0])

# Process remaining segments
for segment in audio_segments[1:]:
    segment_tensor = torch.from_numpy(segment)

    max_amplitude = max(abs(segment_tensor.max()), abs(segment_tensor.min()))
    if max_amplitude > 0:
        segment_tensor /= max_amplitude

    silence_threshold = 0.01  
    final_audio = final_audio[final_audio.abs() > silence_threshold]

    # Concatenate with minimal padding or crossfade
    final_audio = torch.cat([final_audio, segment_tensor], dim=-1)


In [11]:
output_file = "/kaggle/working/final_audio.wav" 
sf.write(output_file, final_audio.numpy(), samplerate=44100)

print(f"Final audio saved as: {output_file}")


Final audio saved as: /kaggle/working/final_audio.wav
